In [1]:
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
search_result_urls = []
for i in range(7):
    if i == 0:
        cur_url = "https://www.homedepot.com/b/Kitchen-Kitchen-Faucets/Delta/N-5yc1vZar0kZg2/Ntk-brandsearch/Ntt-delta%2Bfaucet?NCNI-5"
    else:
        num = 24*i
        cur_url = "https://www.homedepot.com/b/Kitchen-Kitchen-Faucets/Delta/N-5yc1vZar0kZg2/Ntk-brandsearch/Ntt-delta%2Bfaucet?NCNI-5&Nao="+str(num)+"&Ns=None"
    search_result_urls.append(cur_url)
    time.sleep(1)
print(len(search_result_urls))
print(search_result_urls)

7
['https://www.homedepot.com/b/Kitchen-Kitchen-Faucets/Delta/N-5yc1vZar0kZg2/Ntk-brandsearch/Ntt-delta%2Bfaucet?NCNI-5', 'https://www.homedepot.com/b/Kitchen-Kitchen-Faucets/Delta/N-5yc1vZar0kZg2/Ntk-brandsearch/Ntt-delta%2Bfaucet?NCNI-5&Nao=24&Ns=None', 'https://www.homedepot.com/b/Kitchen-Kitchen-Faucets/Delta/N-5yc1vZar0kZg2/Ntk-brandsearch/Ntt-delta%2Bfaucet?NCNI-5&Nao=48&Ns=None', 'https://www.homedepot.com/b/Kitchen-Kitchen-Faucets/Delta/N-5yc1vZar0kZg2/Ntk-brandsearch/Ntt-delta%2Bfaucet?NCNI-5&Nao=72&Ns=None', 'https://www.homedepot.com/b/Kitchen-Kitchen-Faucets/Delta/N-5yc1vZar0kZg2/Ntk-brandsearch/Ntt-delta%2Bfaucet?NCNI-5&Nao=96&Ns=None', 'https://www.homedepot.com/b/Kitchen-Kitchen-Faucets/Delta/N-5yc1vZar0kZg2/Ntk-brandsearch/Ntt-delta%2Bfaucet?NCNI-5&Nao=120&Ns=None', 'https://www.homedepot.com/b/Kitchen-Kitchen-Faucets/Delta/N-5yc1vZar0kZg2/Ntk-brandsearch/Ntt-delta%2Bfaucet?NCNI-5&Nao=144&Ns=None']


In [4]:
#get 24 product links on 7 search result page
product_records = []
product_urls = []
for url in search_result_urls:
    Newpage = urllib.request.urlopen(url)
    PageHtml = Newpage.read().decode('utf-8')
    PageHtml_soup = BeautifulSoup(PageHtml,'html.parser')
    #print(PageHtml_soup.prettify())
    #find one product url
    product_urls_inhtml = PageHtml_soup.find_all('a', attrs = {'data-pod-type':"pr"})
    for product_url_inhtml in product_urls_inhtml:
        product_suffix = product_url_inhtml['href']
        product_url = "https://www.homedepot.com" + product_suffix
        p_id = product_suffix.split('/')[-1]
        record = (p_id, product_url)
        product_urls.append(product_url)
        product_records.append(record)        
    time.sleep(1)
print(len(product_records))
print(len(product_urls))
product_df = pd.DataFrame(product_records, columns=['p_id', 'product_url'])
product_df.to_excel('products.xlsx', index = False)

193
193


In [7]:
#get review for each product
review_records=[]
for product_url in product_urls:
    product_Newpage = urllib.request.urlopen(product_url)
    product_PageHtml = product_Newpage.read().decode('utf-8')
    product_PageHtml_soup = BeautifulSoup(product_PageHtml,'html.parser')
    p_id = product_url.split('/')[-1]
    reviews = product_PageHtml_soup.find_all('div', attrs = {'class':"bvseo-review"})
    for review in reviews:
        itemReviewed = review.find_all('meta', attrs = {'itemprop':"itemReviewed"})
        productname = itemReviewed[0]['content']
        ratingValue_html = review.find_all('span', attrs = {'itemprop':"ratingValue"})
        ratingValue = ratingValue_html[0].text
        author_html = review.find_all('span', attrs = {'itemprop':"author"})
        author = author_html[0].text
        title_html = review.find_all('span', attrs = {'itemprop':"name"})
        title = title_html[0].text
        datePublished = review.find_all('meta', attrs = {'itemprop':"datePublished"})
        date = datePublished[0]['content']
        description_html = review.find_all('span',attrs = {'itemprop':"description"})
        description = description_html[0].text              
        record = (p_id, productname, ratingValue,author,title,date,description)
        review_records.append(record)
    time.sleep(0.2)
#build dataframe and save as excel
#visualize the data
print(len(review_records))
review_df = pd.DataFrame(review_records, columns=['p_id', 'ProductName','RatingValue','Author','Title','Date','Description'])
review_df.to_excel('reviews.xlsx', index = True)

1346
